In [1]:
import os
import pandas as pd
import json
import time

In [7]:
#install requests library
!pip install requests

In [2]:
from __future__ import print_function
import argparse
import json
import pprint
import requests
import sys
import urllib

In [3]:
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [4]:
#apikey, your apikey
API_KEY= 'mTVB4LPIXmpZqi8ZNBzRvdFt21huxCiIOEoT87fZNlqobi6A-GaXh5jp6GPl993oImBhRSG1vxOI5aGcEUzxK66_GI3CZfs4pzW3auH16lvWMXJSCr-dz34NQGy6XHYx'
API_KEY2='YeZVTfwaEZN_evAjzsF0u3a6sFX65uj47Sggd_IuPN7J1R6sM_E1H-IyKoK8OjiW3U4VG1_jkxlMxFVj5l9QqWJx9vdk177h4sJpIVF9CXLtombRYrqYezSWFLO6XHYx'

In [7]:
# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.

In [12]:
def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [13]:
def searchlatlongall(api_key, term, lat, long):
    url_params = {
        'term': term.replace(' ', '+'),
        'latitude':lat,        
        'longitude': long,
        'limit': 50,
        'radius':1700 #slightly over 1 mile
    }
    response = request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)
    return(response)

In [25]:
latlongdata=pd.read_excel('latlongoutputreduced2.xlsx')
latlongdata=latlongdata2.reset_index(drop=True)

In [26]:
len(latlongdata)

10432

In [ ]:
countall=[]
# The countall table helps us pick out the coordinates, around which there are more than 50 coffee shops. We need to make more requests for these coordinates to get all the records.
for x in range(len(latlongdata)):
    print(x)
    #We try 5 times, if it still does not work, we break
    for attempt in range(5):
        try:
            if x<4500:
                response=searchlatlongall(API_KEY,'coffee',latlongdata['lat'][x],latlongdata['long'][x])
            else:
                response=searchlatlongall(API_KEY2,'coffee',latlongdata['lat'][x],latlongdata['long'][x])
            count=response['total']
            if not count:
                count=0  
        except Exception:
            time.sleep(5.0)
            continue
        else:
            break
    else:
        print("we failed all the attempts at x="+str(x)+" - deal with the consequences!")       
    if count>0:
        #if there is at least one coffee shop, we print the response body.
        with open(str(x)+'data.txt', 'w', encoding='utf-8') as f:
          json.dump(response, f, ensure_ascii=False)
    countall.append((x,count))
    time.sleep(2.0)
pd.DataFrame(countall,columns=('x','Count')).to_csv('countall.csv')